In [1]:
from pathlib import Path
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Path to project root
PROJECT_ROOT = Path.home() / "Desktop" / "Insurance_Project"

# Add root to Python path
sys.path.append(str(PROJECT_ROOT))

In [2]:
from pathlib import Path
import os
import importlib
from src.data import preprocessing as pr
import re
file_path = PROJECT_ROOT/"data"/"raw"/"insurance_claims.csv"
data = pr.load_data(file_path)
# Feature engineering specific to the prototype dataset
data = data.drop('_c39', axis=1)
data.replace('?', np.nan, inplace=True)

for char in ['Y', 'YES']:
    data.replace(char, True, inplace=True)
for char in ['N', 'NO']:
    data.replace(char, False, inplace=True)


C:\Users\phili\AppData\Local\Temp\ipykernel_40200\438751034.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace(char, False, inplace=True)


In [3]:
null_count = int(data.isna().sum().sum()) 
print(f"Null count: {null_count}")

Null count: 972


In [4]:
missing_values = pd.DataFrame(data.isnull().sum().sort_values(ascending=False)) # type: ignore
missing_values

,0
property_damage,360
police_report_available,343
collision_type,178
authorities_contacted,91
age,0
policy_number,0
policy_deductable,0
policy_csl,0
policy_state,0
policy_bind_date,0


In [5]:
target = 'total_claim_amount'
X_train, X_test, X_val, y_train, y_test, y_val = pr.split_data(data, target)# type: ignore

In [6]:
data['policy_bind_date'] = pd.to_datetime(data['policy_bind_date'], yearfirst=True)
data['month'] = data['policy_bind_date'].dt.month
data['month_sin'] = np.sin(2 * np.pi * data['month']/12)
data['month_cos'] = np.cos(2 * np.pi * data['month']/12)

data = data.drop('month', axis=1)

In [7]:

data['insured_zip'] = data['insured_zip'].astype(object)
date_cols = data.select_dtypes(include="datetime").columns# type: ignore
bool_cols = []
for col in data.columns:
    unique_vals = set(data[col].dropna().unique())
    if unique_vals.issubset({True, False}):
        data[col] = data[col].astype('boolean')  # nullable bool
        bool_cols.append(col)

for col in bool_cols:
    print(f"Bool Column --------- {col}")
print("\n----------------------------------------------------\n")

for col in date_cols:
    print(f"Date Column --------- {col}")
print("\n----------------------------------------------------\n")
num_cols = data.loc[:,~data.columns.isin(date_cols)].select_dtypes(include="number").columns# type: ignore
for col in num_cols:
    print(f"Numerical Column --------- {col}")

print("\n----------------------------------------------------\n")
cat_cols = data.select_dtypes(include="object").columns# type: ignore
cat_cols = [x for x in cat_cols if x not in bool_cols]
for col in cat_cols:
    print(f"Categorical Column --------- {col}")

num_cols.drop("policy_number")

data = data.drop('policy_bind_date', axis=1)
# Cyclical interpretation of datetime feature

Bool Column --------- property_damage
Bool Column --------- police_report_available
Bool Column --------- fraud_reported

----------------------------------------------------

Date Column --------- policy_bind_date

----------------------------------------------------

Numerical Column --------- age
Numerical Column --------- policy_number
Numerical Column --------- policy_deductable
Numerical Column --------- policy_annual_premium
Numerical Column --------- umbrella_limit
Numerical Column --------- capital-gains
Numerical Column --------- capital-loss
Numerical Column --------- incident_hour_of_the_day
Numerical Column --------- number_of_vehicles_involved
Numerical Column --------- bodily_injuries
Numerical Column --------- witnesses
Numerical Column --------- total_claim_amount
Numerical Column --------- injury_claim
Numerical Column --------- property_claim
Numerical Column --------- vehicle_claim
Numerical Column --------- auto_year
Numerical Column --------- month_sin
Numerical C

In [8]:
print(X_train)

                    age  policy_number policy_bind_date policy_state  \
months_as_customer                                                     
239                  41         743092       2013-11-11           OH   
108                  31         492224       2005-12-09           IN   
116                  30         996253       2001-11-29           IN   
8                    21         355085       2012-10-09           IN   
161                  38         192524       2004-01-02           IL   
...                 ...            ...              ...          ...   
464                  61         632627       1990-10-07           OH   
369                  55         577810       2013-04-15           OH   
230                  42         175960       2004-11-16           IN   
102                  28         810189       1999-08-29           OH   
279                  41         389238       2001-06-06           IL   

                   policy_csl  policy_deductable  policy_annual

In [9]:
# data = data.drop('policy_number', axis=1)
# data = data.drop('incident_city', axis=1)
# data = data.drop('incident_location', axis=1)
# data = data.drop('insured_zip', axis=1)
# data = data.drop('insured_hobbies', axis=1)
# data = data.drop('injury_claim', axis=1)
# data = data.drop('property_claim', axis=1)
# data = data.drop('vehicle_claim', axis=1)
# data = data.drop('incident_date', axis=1)




In [10]:

# numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
# ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level', 'incident_type','incident_severity',]
# ordinal_categories = [['OH', 'IN', 'IL'],
#                       ['100/300', '250/500', '500/1000'],
#                       ['MALE', 'FEMALE'],
#                       ['High School', 'Masters','JD', 'MD'],
#                       ['Single Vehicle Collision', 'Multi-vehicle Collision'],
#                       ['Minor Damage', 'Major Damage', 'Total Loss']]
# nominal_columns = [x for x in cat_cols if x not in ordinal_columns]


In [11]:
data = data.drop('policy_number', axis=1)
data = data.drop('insured_hobbies', axis=1)
data = data.drop('injury_claim', axis=1)
data = data.drop('property_claim', axis=1)
data = data.drop('vehicle_claim', axis=1)
data = data.drop('incident_date', axis=1)
data = data.drop('collision_type', axis=1)
data = data.drop('number_of_vehicles_involved', axis=1)
data = data.drop('incident_severity', axis=1)
data = data.drop('incident_type', axis=1)
data = data.drop('bodily_injuries', axis=1)
data = data.drop('property_damage', axis=1)
data = data.drop('police_report_available', axis=1)
data = data.drop('authorities_contacted', axis=1)
data = data.drop('incident_hour_of_the_day', axis=1)
data = data.drop('fraud_reported', axis=1)
data = data.drop('capital-gains', axis=1)
data = data.drop('capital-loss', axis=1)



In [12]:
numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level']
ordinal_categories = [['OH', 'IN', 'IL'],
                      ['100/300', '250/500', '500/1000'],
                      ['MALE', 'FEMALE'],
                      ['High School', 'Masters','JD', 'MD'],]
nominal_columns = [x for x in cat_cols if x not in ordinal_columns]


In [13]:
print(numerical_columns)

Index(['age', 'policy_number', 'policy_deductable', 'policy_annual_premium',
       'umbrella_limit', 'insured_zip', 'capital-gains', 'capital-loss',
       'incident_hour_of_the_day', 'number_of_vehicles_involved',
       'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim', 'auto_year'],
      dtype='object')


In [14]:
importlib.reload(pr)
preprocessor, X_train_processed = pr.prepreprocessing_pipeline(
    X = X_train,
    numerical_cols=numerical_columns,
    ordinal_cols=ordinal_columns,
    ordinal_categories=ordinal_categories,
    nominal_cols=nominal_columns,
    bool_cols=bool_cols
)

In [15]:
importlib.reload(pr)
global_mean = y_train.mean()
X_val_processed = pr.apply_preprocessing_pipeline(X_val,
                                                  preprocessor=preprocessor)

X_test_processed = pr.apply_preprocessing_pipeline(X_test,
                                                  preprocessor=preprocessor)

In [16]:
importlib.reload(pr)
X_train_processed, y_train = pr.remove_null(X_train_processed, y_train)
X_val_processed, y_val = pr.remove_null(X_val_processed, y_val)
X_test_processed, y_test = pr.remove_null(X_test_processed, y_test)

In [17]:
train_data = pd.DataFrame(X_train_processed)
test_data = pd.DataFrame(X_test_processed)
val_data = pd.DataFrame(X_val_processed)
processed_data = pd.concat([
    train_data,
    test_data,
    val_data
], ignore_index=True)   

In [18]:
processed_data.shape

(1000, 37)

In [19]:
print(processed_data.isnull().sum().sum())

0


In [20]:
i = 0
for df in [train_data, test_data, val_data]:
    print(f"Null values in {i}: {df.isnull().sum().sum()}")
    i += 1

Null values in 0: 0
Null values in 1: 0
Null values in 2: 0


In [21]:
y_train_data = pd.Series(y_train)
y_test_data = pd.Series(y_test)
y_val_data = pd.Series(y_val)

In [22]:
y_train_data

months_as_customer
239    10790
108    57330
116    59670
8      91260
161    60480
       ...  
464    79800
369    85300
230    58850
102    73400
279    28800
Name: total_claim_amount, Length: 700, dtype: int64

In [ ]:
data_folder = PROJECT_ROOT/"data"/"processed"
cluster_folder = PROJECT_ROOT/"data"/"processed"/"indexed"
train_data.to_csv(data_folder/"X_train.csv", index=False)
test_data.to_csv(data_folder/"X_test.csv", index=False)
val_data.to_csv(data_folder/"X_val.csv",index=False)
y_train_data.to_csv(data_folder/"y_train.csv",index=False)
y_test_data.to_csv(data_folder/"y_test.csv",index=False)
y_val_data.to_csv(data_folder/"y_val.csv",index=False)

#Cluster data
train_data.to_csv(cluster_folder/"X_train.csv", index=True)
test_data.to_csv(cluster_folder/"X_test.csv", index=True)

In [25]:
# numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
# ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level', 'incident_type','incident_severity',]
# ordinal_categories = [['OH', 'IN', 'IL'],
#                       ['100/300', '250/500', '500/1000'],
#                       ['MALE', 'FEMALE'],
#                       ['High School', 'Masters','JD', 'MD'],
#                       ['Single Vehicle Collision', 'Multi-vehicle Collision'],
#                       ['Minor Damage', 'Major Damage', 'Total Loss']]
# nominal_columns = [x for x in cat_cols if x not in ordinal_columns]


In [26]:
numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level']
ordinal_categories = [['OH', 'IN', 'IL'],
                      ['100/300', '250/500', '500/1000'],
                      ['MALE', 'FEMALE'],
                      ['High School', 'Masters','JD', 'MD'],]
nominal_columns = [x for x in cat_cols if x not in ordinal_columns]

In [27]:
numerical_columns = [col for col in numerical_columns if col in data.columns]
ordinal_columns = [col for col in ordinal_columns if col in data.columns]
nominal_columns = [col for col in nominal_columns if col in data.columns]
bool_cols = [col for col in bool_cols if col in data.columns]

In [28]:
print(numerical_columns)

['age', 'policy_deductable', 'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'witnesses', 'auto_year']


In [29]:
from sklearn.model_selection import train_test_split
target = 'total_claim_amount'
X, y = data[[col for col in data.columns if col not in target]], data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

preprocessor, X_train_processed = pr.prepreprocessing_pipeline(
    X = X_train,
    numerical_cols=numerical_columns,
    ordinal_cols=ordinal_columns,
    ordinal_categories=ordinal_categories,
    nominal_cols=nominal_columns,
    bool_cols=bool_cols
)
#X_test_processed = pr.apply_preprocessing_pipeline(X_test, preprocessor)
train_reg, test_reg = pd.DataFrame(X_train_processed), pd.DataFrame(X_test_processed)
y_train_reg, y_test_reg = pd.Series(y_train), pd.Series(y_test)

train_reg.to_csv(data_folder/"reg"/"X_train.csv", index=False)
test_reg.to_csv(data_folder/"reg"/"X_test.csv", index=False)
y_train_reg.to_csv(data_folder/"reg"/"y_train.csv", index=False)
y_test_reg.to_csv(data_folder/"reg"/"y_test.csv", index=False)


In [30]:
(data['total_claim_amount'] >= 20000).value_counts(normalize=True)


total_claim_amount
True     0.82
False    0.18
Name: proportion, dtype: float64

In [ ]:
importlib.reload(pr)

target = 'total_claim_amount'

X_train, X_test, y_train, y_test = pr.split_data_for_classification(data, target)

preprocessor, X_train_processed = pr.prepreprocessing_pipeline(
    X = X_train,
    numerical_cols=numerical_columns,
    ordinal_cols=ordinal_columns,
    ordinal_categories=ordinal_categories,
    nominal_cols=nominal_columns,
    bool_cols=bool_cols
)
X_train_processed, y_train = pr.oversampling(X_train_processed, y_train)
X_test_processed = pr.apply_preprocessing_pipeline(X_test, preprocessor)
train_class, test_class = pd.DataFrame(X_train_processed), pd.DataFrame(X_test_processed)
y_train_class, y_test_class = pd.Series(y_train), pd.Series(y_test)

train_class.to_csv(data_folder/"classification"/"X_train.csv", index=False)
test_class.to_csv(data_folder/"classification"/"X_test.csv", index=False)
y_train_class.to_csv(data_folder/"classification"/"y_train.csv", index=False)
y_test_class.to_csv(data_folder/"classification"/"y_test.csv", index=False)


severe
0    0.799
1    0.201
Name: proportion, dtype: float64


In [32]:
y_train_class.value_counts(normalize=True)

severe
0    0.5
1    0.5
Name: proportion, dtype: float64

In [33]:
y_test_class.value_counts(normalize=True)

severe
0    0.8
1    0.2
Name: proportion, dtype: float64

In [34]:
print(X.shape)

(1000, 20)


In [35]:
features = X.columns
features

Index(['age', 'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_relationship',
       'incident_state', 'incident_city', 'incident_location', 'witnesses',
       'auto_make', 'auto_model', 'auto_year', 'month_sin', 'month_cos'],
      dtype='object')

In [36]:
X_train = pd.DataFrame(X_train_processed)

In [37]:
X_test = pd.DataFrame(X_test_processed)

In [38]:
X_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.686729,1.414868,0.308389,2.316550,1.695667,-1.345233,0.953545,1.0,1.0,1.0,3.0,0.0,37.0,82.0,150.0,97.0,0.0,50.0,26.0
1,-1.102792,-0.211417,-1.107000,-0.454075,-0.282159,1.345233,-0.376210,1.0,1.0,1.0,-1.0,0.0,43.0,98.0,150.0,88.0,0.0,47.0,19.0
2,-0.990947,-0.211417,0.721864,-0.454075,1.605387,-1.345233,-0.376210,1.0,0.0,1.0,-1.0,0.0,50.0,103.0,150.0,65.0,0.0,44.0,20.0
3,0.127503,-1.024560,-0.070905,-0.454075,-0.438129,-0.448411,-1.207307,0.0,0.0,1.0,0.0,0.0,50.0,103.0,132.0,94.0,0.0,48.0,16.0
4,-1.550173,-0.211417,0.530784,-0.454075,-0.385204,1.345233,0.953545,2.0,2.0,1.0,-1.0,0.0,41.0,103.0,132.0,94.0,0.0,48.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,-1.438328,-1.024560,0.969209,2.316550,1.597574,0.448411,-0.209991,2.0,2.0,0.0,-1.0,0.0,53.0,98.0,68.0,65.0,0.0,33.0,11.0
396,0.127503,1.414868,0.045127,1.854779,1.447440,-0.448411,-0.043771,0.0,2.0,1.0,1.0,0.0,42.0,98.0,68.0,65.0,0.0,33.0,11.0
397,-0.319877,-0.211417,-1.221719,-0.454075,-0.720065,-1.345233,0.454887,2.0,0.0,1.0,1.0,0.0,42.0,104.0,68.0,94.0,0.0,40.0,11.0
398,1.134109,-0.211417,0.089972,-0.454075,-0.704424,-0.448411,-0.209991,1.0,2.0,0.0,3.0,0.0,37.0,98.0,68.0,84.0,0.0,43.0,17.0


In [39]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1.245954,-1.024560,-0.231305,1.393008,1.680462,-0.448411,-1.705965,2.000000,1.000000,0.000000,-1.000000,1.0,43.000000,98.000000,150.000000,89.000000,1.0,40.000000,18.000000
1,-0.319877,1.414868,0.356974,-0.454075,1.569884,1.345233,0.288668,1.000000,0.000000,1.000000,-1.000000,1.0,54.000000,98.000000,150.000000,88.000000,1.0,44.000000,13.000000
2,-1.102792,-1.024560,0.499030,1.854779,-0.936787,0.448411,0.454887,2.000000,1.000000,0.000000,0.000000,1.0,44.000000,104.000000,150.000000,88.000000,1.0,41.000000,26.000000
3,0.686729,1.414868,-0.139506,-0.454075,-0.471977,-0.448411,0.122448,0.000000,1.000000,1.000000,1.000000,1.0,31.000000,98.000000,132.000000,84.000000,1.0,35.000000,8.000000
4,0.127503,1.414868,2.031566,-0.454075,-0.655679,-0.448411,-1.373527,0.000000,1.000000,0.000000,2.000000,1.0,54.000000,103.000000,150.000000,97.000000,1.0,41.000000,18.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,1.555868,1.371882,-1.371453,-0.454075,-0.493944,0.432608,-0.064274,0.017621,1.017621,1.000000,0.017621,1.0,40.911893,103.017621,132.000000,83.000000,1.0,47.929514,16.070486
954,0.126569,1.414868,0.749838,-0.454075,1.430638,-0.448411,-0.025148,0.000000,0.112039,0.887961,-1.000000,1.0,34.896311,82.000000,132.000000,83.887961,1.0,47.112039,18.215728
955,1.741430,-1.024560,0.399818,-0.454075,-0.593186,0.448411,-0.440559,1.397856,1.397856,0.397856,0.000000,1.0,51.000000,98.000000,67.397856,95.806433,1.0,44.817155,14.806433
956,0.283738,-0.950085,-1.145904,-0.454075,-0.625221,-0.475791,0.298817,1.000000,0.000000,0.969470,1.969470,1.0,50.000000,97.511527,150.000000,84.000000,1.0,44.183177,13.000000


In [40]:
importlib.reload(pr)
processed_data = pr.clean_data(data=data)

In [41]:
print(processed_data.isna().sum().sum())

0


In [42]:
processed_data.to_csv(PROJECT_ROOT/"dashboard"/"assets"/"dashboard_data.csv")

In [43]:
processed_data

,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,...,incident_city,incident_location,witnesses,total_claim_amount,auto_make,auto_model,auto_year,month_sin,month_cos,severe
months_as_customer,,,,,,,,,,,,,,,,,,,,,
328,48.0,OH,250/500,1000.0,1406.91,0.0,466132,MALE,MD,craft-repair,...,Columbus,9935 4th Drive,2.0,71610.0,Saab,92x,2004.0,-8.660254e-01,5.000000e-01,0
228,42.0,IN,250/500,2000.0,1197.22,5000000.0,468176,MALE,MD,machine-op-inspct,...,Riverwood,6608 MLK Hwy,0.0,5070.0,Mercedes,E400,2007.0,1.224647e-16,-1.000000e+00,0
134,29.0,OH,100/300,2000.0,1413.14,5000000.0,430632,FEMALE,PhD,sales,...,Columbus,7121 Francis Lane,3.0,34650.0,Dodge,RAM,2007.0,-1.000000e+00,-1.836970e-16,0
256,41.0,IL,250/500,2000.0,1415.74,6000000.0,608117,FEMALE,PhD,armed-forces,...,Arlington,6956 Maple Drive,2.0,63400.0,Chevrolet,Tahoe,2014.0,5.000000e-01,-8.660254e-01,0
228,44.0,IL,500/1000,1000.0,1583.91,6000000.0,610706,MALE,Associate,sales,...,Arlington,3041 3rd Ave,1.0,6500.0,Accura,RSX,2009.0,1.224647e-16,-1.000000e+00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,38.0,OH,500/1000,1000.0,1310.80,0.0,431289,FEMALE,Masters,craft-repair,...,Northbrook,6045 Andromedia St,1.0,87200.0,Honda,Accord,2006.0,-5.000000e-01,-8.660254e-01,1
285,41.0,IL,100/300,1000.0,1436.79,0.0,608177,FEMALE,PhD,prof-specialty,...,Northbend,3092 Texas Drive,3.0,108480.0,Volkswagen,Passat,2015.0,5.000000e-01,8.660254e-01,1
130,34.0,OH,250/500,500.0,1383.49,3000000.0,442797,FEMALE,Masters,armed-forces,...,Arlington,7629 5th St,3.0,67500.0,Suburu,Impreza,1996.0,8.660254e-01,5.000000e-01,0


In [44]:
importlib.reload(pr)

<module 'src.data.preprocessing' from 'C:\\Users\\phili\\Desktop\\Insurance_Project\\src\\data\\preprocessing.py'>

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

preprocessor_pickle = pr.unfit_prepreprocessing_pipeline(
    numerical_cols=numerical_columns,
    ordinal_cols=ordinal_columns,
    ordinal_categories=ordinal_categories,
    nominal_cols=nominal_columns,
    bool_cols=bool_cols)
preprocessor_pickle.fit(X)

,transformers,"[('num', ...), ('ord', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [46]:
import joblib
joblib.dump(preprocessor_pickle, PROJECT_ROOT/"models"/"preprocessor.pkl")
joblib.dump(features, PROJECT_ROOT/"models"/"features.pkl")


['C:\\Users\\phili\\Desktop\\Insurance_Project\\models\\features.pkl']